In [17]:
import json, random, re, collections, itertools, base64, hashlib
from datetime import datetime, timedelta
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(x))
annot_path = Path("../annotations")
data_path = Path("../data")

In [43]:
def dt2days(lst):
    events = list(map(at("start", "end", "action", "resource","text") ,lst))
    ret = collections.defaultdict(list)
    for start,end,action, resource, text in events:
        start = (datetime.strptime(start, "%Y-%m-%dT00:00:00") - datetime(2020,1,1)).days
        end = (datetime.strptime(end, "%Y-%m-%dT00:00:00") - datetime(2020,1,1)).days
        for i in range(start, end):
            ret[resource].append((i, action, text))
    return  dict(ret)
annotations = dict()
for p in annot_path.iterdir():
    annotaion_id = p.name.split('.',1)[0]
    with p.open('r') as f:
        annotation = json.load(f)
    if int(annotation["status"])<=0:
        continue
    annotations[annotaion_id]=list(map(dt2days,annotation["labels"]))
with (data_path/"annotaions.json").open('w') as f:
    json.dump(annotations, f)
annotations

{'104786': [{'GW4': [(0, 'TPOT', 'Pot'),
    (0, 'IfDyZlP3_', 'frozen chopped spinach'),
    (0, 'IlGA3C7DK', 'water'),
    (0, 'Iz0wiMjVJ', 'onion'),
    (0, 'IvZ0DqVmo', 'garlic powder')]},
  {'GW4': [(0, 'TPOT', 'Pot'),
    (0, 'IfDyZlP3_', 'frozen chopped spinach'),
    (0, 'IlGA3C7DK', 'water'),
    (0, 'Iz0wiMjVJ', 'onion'),
    (0, 'IvZ0DqVmo', 'garlic powder')],
   'GW2': [(1, 'TPOT', 'Pot'),
    (2, 'TPOT', 'Pot'),
    (3, 'TPOT', 'Pot'),
    (4, 'TPOT', 'Pot'),
    (1, 'IfDyZlP3_', 'frozen chopped spinach'),
    (2, 'IfDyZlP3_', 'frozen chopped spinach'),
    (3, 'IfDyZlP3_', 'frozen chopped spinach'),
    (4, 'IfDyZlP3_', 'frozen chopped spinach'),
    (1, 'IlGA3C7DK', 'water'),
    (2, 'IlGA3C7DK', 'water'),
    (3, 'IlGA3C7DK', 'water'),
    (4, 'IlGA3C7DK', 'water'),
    (1, 'Iz0wiMjVJ', 'onion'),
    (2, 'Iz0wiMjVJ', 'onion'),
    (3, 'Iz0wiMjVJ', 'onion'),
    (4, 'Iz0wiMjVJ', 'onion'),
    (1, 'IvZ0DqVmo', 'garlic powder'),
    (2, 'IvZ0DqVmo', 'garlic powder'),
    (3

0